<a href="https://colab.research.google.com/github/francesco-mollica/word2vec-entity_embeddings/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"

# CHOI_path = base_dir + "corpus_data_CHOI.txt"
# FIGER_path = base_dir + "corpus_data_FIGER.txt"
# ontonotes_path = base_dir + "corpus_data_ontonotes.txt"
balanced_ontonotes_path = base_dir + "corpus_data_balanced_ontonotes.txt"
# bbn_path = base_dir + "corpus_data_bbn.txt"

Mounted at /content/drive/


In [40]:
restrict_list_name = 'restricted_list_balanced_ontonotes.pkl'
with open(base_dir + restrict_list_name, 'rb') as f:
  restricted_word_set = pickle.load(f)

In [41]:
from gensim.models import Word2Vec

In [42]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
            for line in open(base_dir + self.filename):
                yield line.split()
 
 # a memory-friendly iterator

In [43]:
sentences = MySentences("corpus_data_balanced_ontonotes.txt")
w2v_model = Word2Vec(iter=1, min_count=1)

In [44]:
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(3375, 8850)

In [45]:
# execute one time a most_similar otherwise vectors_norm in restrict_w2v should be equal to None
w2v_model.wv.most_similar(positive=["/PERSON"])

[(',', 0.5099198222160339),
 ('/LOCATION', 0.4596233367919922),
 ('somebody', 0.4574718177318573),
 ('.', 0.4479295611381531),
 ('to', 0.44764119386672974),
 ('you', 0.44708484411239624),
 ('went', 0.43475139141082764),
 ('that', 0.4342912435531616),
 ('much', 0.42919182777404785),
 ('the', 0.4197062849998474)]

In [46]:
import numpy as np
def restrict_w2v(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.wv.vocab)):
        word = w2v.wv.index2entity[i]
        vec = w2v.wv.vectors[i]
        vocab = w2v.wv.vocab[word]
        vec_norm = w2v.wv.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            new_vectors_norm.append(vec_norm)

    w2v.wv.vocab = new_vocab
    w2v.wv.vectors = np.array(new_vectors)
    w2v.wv.index2entity = np.array(new_index2entity)
    w2v.wv.index2word = np.array(new_index2entity)
    w2v.wv.vectors_norm = np.array(new_vectors_norm)

In [47]:
restrict_w2v(w2v_model, restrict_list)

In [48]:
w2v_model.wv.most_similar(positive=["/LOCATION"])

[('/PERSON', 0.4596233069896698), ('/PERSON/TITLE', 0.3136698007583618)]